In [1]:
# just importing a random selection of things; we can fix this later
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV

from pprint import pprint 

from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from random import seed
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier
from itertools import product
from collections import OrderedDict
from itertools import product

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from __future__ import print_function # For Python 2 / 3 compatability
from sklearn.metrics import confusion_matrix
import itertools

import warnings
warnings.filterwarnings("ignore")

/Users/rebeccastern/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/rebeccastern/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# read in the diagnosis data and the gene expression profile data

# URL: https://ida.loni.usc.edu/pages/access/studyData.jsp?loginKey=1118812361619923667&userEmail=rebecca_stern%40g.harvard.edu&project=ADNI&page=DOWNLOADS&subPage=STUDY_DATA
diagnosis_DXSUM_PDXCONV_ADNIALL = pd.read_csv('Diagnosis/DXSUM_PDXCONV_ADNIALL.csv')
diagnosis_ADSXLIST = pd.read_csv('Diagnosis/ADSXLIST.csv')
diagnosis_BLCHANGE = pd.read_csv('Diagnosis/BLCHANGE.csv')

# URL: https://ida.loni.usc.edu/pages/access/geneticData.jsp#207
ADNI_Gene_Expression_Profile_DICT = pd.read_csv('ADNI_Gene_Expression_Profile/ADNI_Gene_Expression_Profile_DICT.csv')
ADNI_Gene_Expression_Profile = pd.read_csv('ADNI_Gene_Expression_Profile/ADNI_Gene_Expression_Profile.csv')

In [3]:
# view the diagnostic data - shape and top few rows, etc. 

print('shape of diagnosis_DXSUM_PDXCONV_ADNIALL: ', diagnosis_DXSUM_PDXCONV_ADNIALL.shape)
print('shape of diagnosis_ADSXLIST: ', diagnosis_ADSXLIST.shape)
print('shape of diagnosis_BLCHANGE: ', diagnosis_BLCHANGE.shape)

diagnosis_DXSUM_PDXCONV_ADNIALL.columns.values
# diagnosis_DXSUM_PDXCONV_ADNIALL['DXAD']
# diagnosis_ADSXLIST.head()
diagnosis_BLCHANGE.head()

shape of diagnosis_DXSUM_PDXCONV_ADNIALL:  (10291, 53)
shape of diagnosis_ADSXLIST:  (4884, 39)
shape of diagnosis_BLCHANGE:  (10284, 27)


,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,BCPREDX,...,BCCDR,BCDEPRES,BCSTROKE,BCDELIR,BCEXTCIR,BCEXTSP,BCCORADL,BCCORCOG,BCSUMM,update_stamp
0,ADNI1,2,7,10,bl,bl,2005-10-13,NaN,2005-10-06,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,Patient has a three year history of gradually ...,2005-10-13 00:00:00.0
1,ADNI1,4,15,4,bl,bl,2005-10-19,NaN,2005-10-19,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,The subject has no complaints of memory proble...,2005-10-19 00:00:00.0
2,ADNI1,6,30,11,bl,bl,2005-10-20,NaN,2005-10-20,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,MCI-stable. Developed severe constipation/abd...,2005-10-20 00:00:00.0
3,ADNI1,8,31,11,bl,bl,2005-10-24,NaN,2005-10-24,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,Few minor symptoms.,2005-10-24 00:00:00.0
4,ADNI1,10,14,10,bl,bl,2005-11-08,NaN,2005-11-04,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,A 78 year old hispanic female with no memory c...,2005-11-08 00:00:00.0


In [4]:
diagnosis_DXSUM_PDXCONV_ADNIALL.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,DXCHANGE,...,DXPATYP,DXPOTHSP,DXDEP,DXDEPSP,DXOTHDEM,DXODES,DXOOTHSP,DXCONFID,DIAGNOSIS,update_stamp
0,ADNI1,2.0,2,107,bl,bl,2005-10-01,NaN,2005-09-29,NaN,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN,2005-10-01 00:00:00.0
1,ADNI1,4.0,3,107,bl,bl,2005-10-01,NaN,2005-09-30,NaN,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN,2005-10-01 00:00:00.0
2,ADNI1,6.0,5,107,bl,bl,2005-10-01,NaN,2005-09-30,NaN,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN,2005-10-01 00:00:00.0
3,ADNI1,8.0,8,107,bl,bl,2005-10-01,NaN,2005-09-30,NaN,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN,2005-10-01 00:00:00.0
4,ADNI1,10.0,7,10,bl,bl,2005-10-06,NaN,2005-10-06,NaN,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN,2005-10-06 00:00:00.0


In [5]:
# Print the list of all unique ID's in each diagnosis dataset
print('There are this many unique patient RIDs in {}: {}.' \
      .format('diagnosis_DXSUM_PDXCONV_ADNIALL', len(diagnosis_DXSUM_PDXCONV_ADNIALL['RID'].unique())))

print('There are this many unique patient RIDs in {}: {}.' \
      .format('diagnosis_ADSXLIST', len(diagnosis_ADSXLIST['RID'].unique())))

print('There are this many unique patient RIDs in {}: {}.' \
      .format('diagnosis_BLCHANGE', len(diagnosis_BLCHANGE['RID'].unique())))

# 2098+916+2093 # this = 5107

There are this many unique patient RIDs in diagnosis_DXSUM_PDXCONV_ADNIALL: 2098.
There are this many unique patient RIDs in diagnosis_ADSXLIST: 916.
There are this many unique patient RIDs in diagnosis_BLCHANGE: 2093.


In [6]:
# load the dataset containing explanation of the variables in the diagnosis datasets
# downloaded from: https://ida.loni.usc.edu/pages/access/studyData.jsp?categoryId=16&subCategoryId=43
ADNI_diagnosis_data_description = pd.read_csv('ADNIMERGE_DICT.csv')
print(ADNI_diagnosis_data_description['FLDNAME'].unique())
ADNI_diagnosis_data_description[['FLDNAME','TEXT']]

['RID' 'PTID' 'VISCODE' 'SITE' 'COLPROT' 'ORIGPROT' 'EXAMDATE' 'DX_bl'
 'AGE' 'PTGENDER' 'PTEDUCAT' 'PTETHCAT' 'PTRACCAT' 'PTMARRY' 'APOE4' 'FDG'
 'PIB' 'AV45' 'CDRSB' 'ADAS11' 'ADAS13' 'MMSE' 'RAVLT' 'RAVLT_immediate'
 'FAQ' 'MOCA' 'EcogPtMem' 'EcogPtLang' 'EcogPtVisspat' 'EcogPtPlan'
 'EcogPtOrgan' 'EcogPtDivatt' 'EcogPtTotal' 'EcogSPMem' 'EcogSPLang'
 'EcogSPVisspat' 'EcogSPPlan' 'EcogSPOrgan' 'EcogSPDivatt' 'EcogSPTotal'
 'Ventricles' 'Hippocampus' 'WholeBrain' 'Entorhinal' 'Fusiform' 'MidTemp'
 'ICV' 'DX' 'EXAMDATE_bl' 'CDRSB_bl' 'ADAS11_bl' 'ADAS13_bl' 'MMSE_bl'
 'RAVLT_bl' 'RAVLT_immediate_bl' 'FAQ_bl' 'Ventricles_bl' 'Hippocampus_bl'
 'WholeBrain_bl' 'Entorhinal_bl' 'Fusiform_bl' 'MidTemp_bl' 'ICV_bl'
 'MOCA_bl' 'EcogPtMem_bl' 'EcogPtLang_bl' 'EcogPtVisspat_bl'
 'EcogPtPlan_bl' 'EcogPtOrgan_bl' 'EcogPtDivatt_bl' 'EcogPtTotal_bl'
 'EcogSPMem_bl' 'EcogSPLang_bl' 'EcogSPVisspat_bl' 'EcogSPPlan_bl'
 'EcogSPOrgan_bl' 'EcogSPDivatt_bl' 'EcogSPTotal_bl' 'FDG_bl' 'PIB_bl'
 'AV45_bl' 'Y

,FLDNAME,TEXT
0,RID,Participant roster ID
1,PTID,Participant ID
2,VISCODE,Visit code
3,SITE,Site
4,COLPROT,Protocol under which data was collected
5,ORIGPROT,Protocol from which subject originated
6,EXAMDATE,Examination Date
7,DX_bl,Baseline Dx
8,AGE,Age at baseline
9,PTGENDER,Sex


In [7]:
print(ADNI_diagnosis_data_description.columns.unique())

Index(['ID', 'FLDNAME', 'TBLNAME', 'CRFNAME', 'QUESTION', 'TEXT', 'STATUS',
       'DEPRULE', 'TYPE', 'LENGTH', 'DECIML', 'RANGEVAL', 'RANGESQL',
       'RANGEPRL', 'UNITS', 'CODE', 'NOTES', 'OBJID', 'COLID', 'RANGEJSCPT',
       'NAVJSCPT', 'FTYPE', 'TAB', 'DE', 'MC', 'VISIBLE', 'LOGVISIBLE',
       'MTBLNAME', 'SUBTBLNAME', 'POPJSCPT', 'GRID', 'DEF', 'update_stamp'],
      dtype='object')


Source link for understanding what each of the columns/values abbreviations mean: 
https://drive.google.com/a/harvard.edu/file/d/11KtDXlEV5b6nLZt18V7GjFh7-BsTbFnc/view?usp=sharing


if you want to download it yourself, go to ADNI site, click "study info" --> "data and databases" 
then download the one titled "key ADNI tables merged into one table - Dictionary"

In [8]:
# Identify the diagnosis column  (DX)
ADNI_diagnosis_data_description[['FLDNAME','TEXT']][ADNI_diagnosis_data_description['FLDNAME']=='DX']

,FLDNAME,TEXT
47,DX,Dx status


In [9]:
print(diagnosis_DXSUM_PDXCONV_ADNIALL.columns.values)
print(diagnosis_DXSUM_PDXCONV_ADNIALL['DXCURREN'].unique())
dfv = diagnosis_DXSUM_PDXCONV_ADNIALL['DXCURREN'].value_counts(dropna=False)
print(dfv)
print(diagnosis_DXSUM_PDXCONV_ADNIALL['DXCURREN'].shape[0])

['Phase' 'ID' 'RID' 'SITEID' 'VISCODE' 'VISCODE2' 'USERDATE' 'USERDATE2'
 'EXAMDATE' 'DXCHANGE' 'DXCURREN' 'DXCONV' 'DXCONTYP' 'DXREV' 'DXNORM'
 'DXNODEP' 'DXMCI' 'DXMDES' 'DXMPTR1' 'DXMPTR2' 'DXMPTR3' 'DXMPTR4'
 'DXMPTR5' 'DXMPTR6' 'DXMDUE' 'DXMOTHET' 'DXMOTHSP' 'DXDSEV' 'DXDDUE'
 'DXAD' 'DXADES' 'DXAPP' 'DXAPROB' 'DXAMETASP' 'DXAOTHRSP' 'DXAPOSS'
 'DXAATYSP' 'DXAMETSP' 'DXAOTHSP' 'DXPARK' 'DXPARKSP' 'DXPDES' 'DXPCOG'
 'DXPATYP' 'DXPOTHSP' 'DXDEP' 'DXDEPSP' 'DXOTHDEM' 'DXODES' 'DXOOTHSP'
 'DXCONFID' 'DIAGNOSIS' 'update_stamp']
[  1.   3.   2.  nan]
NaN     6423
 2.0    1606
 3.0    1132
 1.0    1130
Name: DXCURREN, dtype: int64
10291


<font color = red> Note for group 46: Should we select only the entries in the 3 datasets that overlap with RIDs? The RID is the unique patient identifier according to the TA. 

In [10]:
# Next step: only select the RIDs that overlap across all 3 diagnosis datasets

# first make a list containing all the unique RIDs in each dataset
RIDs_DXSUM = list(diagnosis_DXSUM_PDXCONV_ADNIALL['RID'].unique())
RIDs_ADSXLIST = list(diagnosis_ADSXLIST['RID'].unique())
RIDs_BLCHANGE = list(diagnosis_BLCHANGE['RID'].unique())

<font color = magenta>
# 11.20.17 from here onward - loading the Medical History data

In [11]:
# 112017 - exploring drug effects per reccs from my parents - RAS
ADNI2_ECG = pd.read_csv('data/Medical_History/ADNI2_ECG.csv')
BLSCHECK= pd.read_csv('data/Medical_History/BLSCHECK.csv')
NEUROEXM= pd.read_csv('data/Medical_History/NEUROEXM.csv')
NEUROPATH_03_31_16_DICT = pd.read_csv('data/Medical_History/NEUROPATH_03_31_16_DICT.csv')
NEUROPATH_04_06_17 = pd.read_csv('data/Medical_History/NEUROPATH_04_06_17.csv')
PHYSICAL = pd.read_csv('data/Medical_History/PHYSICAL.csv')
VITALS= pd.read_csv('data/Medical_History/VITALS.csv')
AV45VITALS = pd.read_csv('data/Medical_History/AV45VITALS.csv')
INITHEALTH = pd.read_csv('data/Medical_History/INITHEALTH.csv')
MEDHIST = pd.read_csv('data/Medical_History/MEDHIST.csv')
RECMHIST = pd.read_csv('data/Medical_History/RECMHIST.csv')
BACKMEDS = pd.read_csv('data/Medical_History/BACKMEDS.csv')
RECBLLOG = pd.read_csv('data/Medical_History/RECBLLOG.csv')
AV45FOLLOW = pd.read_csv('data/Medical_History/AV45FOLLOW.csv')
RECCMEDS = pd.read_csv('data/Medical_History/RECCMEDS.csv')
ADVERSE = pd.read_csv('data/Medical_History/ADVERSE.csv')
RECADV = pd.read_csv('data/Medical_History/RECADV.csv')

In [21]:
MEDHIST.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,MHSOURCE,...,MH15BDRUG,MH16SMOK,MH16ASMOK,MH16BSMOK,MH16CSMOK,MH17MALI,MH18SURG,MH19OTHR,MHCOMMEN,update_stamp
0,ADNI1,8,2,107,sc,sc,2005-08-17,NaN,2005-08-17,1,...,NaN,0,NaN,NaN,NaN,0,0,0,-4,2005-08-17 00:00:00.0
1,ADNI1,10,1,10,f,f,2005-08-18,NaN,2005-08-18,1,...,NaN,0,NaN,NaN,NaN,0,0,1,-4,2005-08-18 00:00:00.0
2,ADNI1,12,3,107,sc,sc,2005-08-18,NaN,2005-08-18,1,...,NaN,1,NaN,NaN,NaN,1,1,0,-4,2005-08-18 00:00:00.0
3,ADNI1,14,4,10,sc,sc,2005-08-18,NaN,2005-08-18,1,...,NaN,1,NaN,NaN,NaN,0,1,0,-4,2005-08-18 00:00:00.0
4,ADNI1,16,5,107,sc,sc,2005-08-23,NaN,2005-08-23,1,...,NaN,1,NaN,NaN,NaN,0,1,0,-4,2005-08-23 00:00:00.0


In [14]:
# DATADIC contains 
DATADIC = pd.read_csv('DATADIC.csv')
list(DATADIC['FLDNAME'].unique())
DATADIC[['FLDNAME', 'TEXT']].head()

,FLDNAME,TEXT
0,ID,Record ID
1,SX,NaN
2,USERID,NaN
3,USERDATE,Date record created
4,USERID2,NaN


In [15]:
# checking out the number of records for patients in each of the ADNI phases
ADMCPATIENTDRUGCLASSES_20170512 = pd.read_csv('data/ADMCPATIENTDRUGCLASSES_20170512.csv')
print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI1']))
print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']))
print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI3']))
print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNIGO']))

6191
7417
0
967


In [16]:
# making a list of all the different drugs, appending it as a new column to "ADMCPATIENTDRUGCLASSES_20170512"
print(len(ADMCPATIENTDRUGCLASSES_20170512['RID'].unique()))
ADMCPATIENTDRUGCLASSES_20170512['NA'].head()

NA_LIST = []
for i in range(ADMCPATIENTDRUGCLASSES_20170512.shape[0]): 
    try: 
        NA_LIST.append(ADMCPATIENTDRUGCLASSES_20170512['NA'][i].split(','))
    except:
        NA_LIST.append(ADMCPATIENTDRUGCLASSES_20170512['NA'][i])

ADMCPATIENTDRUGCLASSES_20170512['NA_LIST'] = NA_LIST

ADMCPATIENTDRUGCLASSES_20170512[['RID', 'NA_LIST']].head()

2500


,RID,NA_LIST
0,2,[-4]
1,2,"[MECLIZINE, LEVAQUIN]"
2,2,[-4]
3,2,[VITAMIN D]
4,2,"[GLYBURIDE, PANTOPRAZOLE, PRILOSEC]"


<font color = magenta>
# 11.22.17 from here onward  - ADNIMERGE dataset

In [144]:
# load ADNIMERGE.csv, the main dataset (we will add our own variables to it). 
ADNI_merge = pd.read_csv('ADNIMERGE.csv')
ADNI_merge.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,011_S_0002,bl,11,ADNI1,ADNI1,2005-09-08,CN,74.3,Male,...,NaN,NaN,1.36926,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
1,3,011_S_0003,bl,11,ADNI1,ADNI1,2005-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
2,3,011_S_0003,m06,11,ADNI1,ADNI1,2006-03-13,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.498289,5.96721,6,6,2017-08-13 23:50:48.0
3,3,011_S_0003,m12,11,ADNI1,ADNI1,2006-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.999316,11.96720,12,12,2017-08-13 23:50:48.0
4,3,011_S_0003,m24,11,ADNI1,ADNI1,2007-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,1.998630,23.93440,24,24,2017-08-13 23:50:48.0


In [145]:
''' ADNI2 data within ADNIMERGE.csv '''
# select only the rows that correspond to ADN2
ADNI2_in_ADNI_merge = ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
ADNI2_in_ADNI_merge.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
58,5282,082_S_5282,bl,82,ADNI2,ADNI2,2013-09-09,SMC,66.9,Male,...,1.00000,1.25641,1.13549,NaN,1.326790,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
59,5280,100_S_5280,m24,100,ADNI2,ADNI2,2015-09-29,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,2.031490,24.32790,24,24,2017-10-06 23:20:00.0
68,5280,100_S_5280,m06,100,ADNI2,ADNI2,2014-03-06,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.465435,5.57377,6,6,2017-10-06 23:20:00.0
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,SMC,68.5,Male,...,1.33333,1.37838,1.50629,NaN,0.985156,0.000000,0.00000,0,0,2017-10-06 23:20:00.0


<font color = magenta> 
## NOTE: this is new code (11.23.17) because missing values are also denoted by "-4" and "-1". 
### Please see the training powerpoint for more info: https://adni.loni.usc.edu/wp-content/uploads/2012/08/slide_data_training_part2_reduced-size.pdf

In [272]:
''' missing data: when data = -1 or -4 '''

# calculate the % of each column that is NaNs
nan_series = ADNI2_in_ADNI_merge.isnull().sum()/len(ADNI2_in_ADNI_merge['DX_bl'])
# find only the columns that contain <= 40% NaNs
df2 = nan_series[nan_series <0.4]

# also account for -4 and -1 values (these qualify as missing data)
columns_ADNI2_in_ADNI_merge = ADNI2_in_ADNI_merge.columns.values
minus_4_missingdata = []
minus_1_missingdata = []

new_df = pd.DataFrame()  
count_neg4 = 0
count_neg1 = 0
for col_name in ADNI2_in_ADNI_merge:
    for row in ADNI2_in_ADNI_merge[col_name]:
        if row == -4:
            count_neg4 = count_neg4 + 1
            minus_4_missingdata.append(row)
#             break #Here we want to 'break' out of the inner loop but continue the outer loop
        if row == -1:
            minus_1_missingdata.append(row)
            count_neg1 = count_neg1 + 1
#             break

    new_df[col_name] = (len(minus_4_missingdata),len(minus_1_missingdata))
    
new_df 

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
0,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
1,0,0,0,0,0,0,0,0,0,0,...,265,265,265,265,265,265,265,265,265,265


In [273]:
''' Missing Data continued: we must process NaNs differently than -1 and -4 '''

# convert to strings
ADNI2_in_ADNI_merge_strings = ADNI2_in_ADNI_merge[ADNI2_in_ADNI_merge.columns].astype(str)

new_df2 = pd.DataFrame()

for col_name in ADNI2_in_ADNI_merge_strings:
    count_nan2 = []
    for row in ADNI2_in_ADNI_merge_strings[col_name]:
        if row == 'nan':
            count_nan2.append(row)
    new_df2[col_name] = [count_nan2]
    new_df2[col_name] = len(new_df2[col_name][0])

# append the NaN info to the -4 and -1 missing values dataframe
new_df = new_df.append(new_df2, ignore_index = True) 
new_df.index = ['count: -4','count: -1','count: NaN']
new_df

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
count: -4,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
count: -1,0,0,0,0,0,0,0,0,0,0,...,265,265,265,265,265,265,265,265,265,265
count: NaN,0,0,0,0,0,0,0,0,0,0,...,1665,1544,808,6888,1556,0,0,0,0,0
